In [1]:
import pandas as pd
import plotly.express as px
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, ElasticNet, HuberRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.metrics import root_mean_squared_error, r2_score

## Carregar os Dados já tratados

In [2]:
df_costs = pd.read_csv('../bagging/aula/healthcosts_cleaned.csv')

In [3]:
df_costs.head()

,age,sex,bmi,children,smoker,region,medical charges
0,19,female,27.900,0,1,southwest,16884.92400
1,18,male,33.770,1,0,southeast,1725.55230
2,28,male,33.000,3,0,southeast,4449.46200
3,33,male,22.705,0,0,northwest,21984.47061
4,32,male,28.880,0,0,northwest,3866.85520


In [4]:
df_costs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   age              1338 non-null   int64  
 1   sex              1338 non-null   object 
 2   bmi              1338 non-null   float64
 3   children         1338 non-null   int64  
 4   smoker           1338 non-null   int64  
 5   region           1338 non-null   object 
 6   medical charges  1338 non-null   float64
dtypes: float64(2), int64(3), object(2)
memory usage: 73.3+ KB


## Preparação dos dados

In [5]:
# Preparar os dados para o modelo
X = df_costs.drop(columns=['medical charges'], axis=1)
y = df_costs['medical charges']

In [6]:
# importar preprocessor
import joblib

preprocessor = joblib.load('../bagging/aula/preprocessor_dataset_healthcosts.pkl')

In [9]:
X_train,X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=51)

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)



In [10]:
print(f'Treinamento: {X_train.shape}')
print(f'Teste: {X_test.shape}')

Treinamento: (1070, 10)
Teste: (268, 10)


## Treinamento do Modelo Stacking

In [11]:
# Criar o modelo de Stacking Regressor

# Algoritmos Base
lr_model = LinearRegression()
elastic_model = ElasticNet(random_state=51)
tree_model = DecisionTreeRegressor(random_state=51)

# Meta-modelo ou Meta-learner
huber_model = HuberRegressor()

# Stacking Model
stacking_model = StackingRegressor(
    estimators= [
        ('linear regression', lr_model),
        ('elastic', elastic_model),
        ('decision_tree', tree_model)
    ],
    final_estimator=huber_model,
    # = False Usa apenas as predições dos estimadores
    # = True usa as predições dos estimadores + conjunto de treinamento (dataset)
    passthrough=False
)

In [12]:
# Treinar o modelo
stacking_model.fit(X_train,y_train)

,"estimators estimators: list of (str, estimator)Base estimators which will be stacked together. Each element of thelist is defined as a tuple of string (i.e. name) and an estimatorinstance. An estimator can be set to 'drop' using `set_params`.","[('linear regression', ...), ('elastic', ...), ...]"
,"final_estimator final_estimator: estimator, default=NoneA regressor which will be used to combine the base estimators.The default regressor is a :class:`~sklearn.linear_model.RidgeCV`.",HuberRegressor()
,"cv cv: int, cross-validation generator, iterable, or ""prefit"", default=NoneDetermines the cross-validation splitting strategy used in`cross_val_predict` to train `final_estimator`. Possible inputs forcv are:* None, to use the default 5-fold cross validation,* integer, to specify the number of folds in a (Stratified) KFold,* An object to be used as a cross-validation generator,* An iterable yielding train, test splits,* `""prefit""`, to assume the `estimators` are prefit. In this case, the estimators will not be refitted.For integer/None inputs, if the estimator is a classifier and y iseither binary or multiclass,:class:`~sklearn.model_selection.StratifiedKFold` is used.In all other cases, :class:`~sklearn.model_selection.KFold` is used.These splitters are instantiated with `shuffle=False` so the splitswill be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here.If ""prefit"" is passed, it is assumed that all `estimators` havebeen fitted already. The `final_estimator_` is trained on the `estimators`predictions on the full training set and are **not** cross validatedpredictions. Please note that if the models have been trained on the samedata to train the stacking model, there is a very high risk of overfitting... versionadded:: 1.1 The 'prefit' option was added in 1.1.. note:: A larger number of split will provide no benefits if the number of training samples is large enough. Indeed, the training time will increase. ``cv`` is not used for model evaluation but for prediction.",None
,"n_jobs n_jobs: int, default=NoneThe number of jobs to run in parallel for `fit` of all `estimators`.`None` means 1 unless in a `joblib.parallel_backend` context. -1 meansusing all processors. See :term:`Glossary ` for more details.",None
,"passthrough passthrough: bool, default=FalseWhen False, only the predictions of estimators will be used astraining data for `final_estimator`. When True, the`final_estimator` is trained on the predictions as well as theoriginal training data.",False
,"verbose verbose: int, default=0Verbosity level.",0
,"fit_intercept fit_intercept: bool, default=TrueWhether to calculate the intercept for this model. If setto False, no intercept will be used in calculations(i.e. data is expected to be centered).",True
,"copy_X copy_X: bool, default=TrueIf True, X will be copied; else, it may be overwritten.",True
,"tol tol: float, default=1e-6The precision of the solution (`coef_`) is determined by `tol` whichspecifies a different convergence criterion for the `lsqr` solver.`tol` is set as `atol` and `btol` of :func:`scipy.sparse.linalg.lsqr` whenfitting on sparse training data. This parameter has no effect when fittingon dense data... versionadded:: 1.7",1e-06
,"n_jobs n_jobs: int, default=NoneThe number of jobs to use for the computation. This will only providespeedup in case of sufficiently large problems, that is if firstly`n_targets > 1` and secondly `X` is sparse or if `positive` is setto `True`. ``None`` means 1 unless in a:obj:`joblib.parallel_backend` context. ``-1`` means using allprocessors. See :term:`Glossary ` for more details.",None
,"positive positive: bool, default=FalseWhen set to ``True``, forces the coefficients to be positive. Thisoption is only supported for dense arrays.For a comparison between a linear regression model with positive constraintson the regression coefficients and a linear regression without such constraints,see :ref:`sphx_glr_auto_examp

## Análise de Resultados

In [13]:
# Fazer as predições com base no modelo treinado
y_pred = stacking_model.predict(X_test)

In [14]:
# Mostrar as predições
y_pred

array([ 7695.79959724, 43370.93405492, 20538.53371989, 23306.79628201,
       38542.87992825,  9712.19066584,  7864.68467422, 12191.71869977,
        5530.5934329 ,  9405.74504414,  8707.54848159, 11508.65098615,
        7327.30156712,  2463.04849912,  4553.46990533, 12495.95535168,
        3100.39761183,  6935.23743426, 18344.04697192, 20101.55089448,
        4888.24901415,  6829.38484691, 53567.50549203, 10875.11501712,
        5634.50271863, 15015.90063325, 11475.01694231,  1573.00910166,
       30832.39007449, 19316.03404105,  1437.97164139, 23004.37525715,
        2486.96709226,  2806.40056811,  7060.57244398, 24988.87749613,
        7348.17486445,  1418.96078897, 11657.23837065,  7247.37658504,
       11211.77968941,  1234.95208951,  3676.24582513,  1443.33461361,
       12248.44036693, 12752.260629  , 11556.60219094, 40223.96058083,
        8302.20504108, 12560.89421783,  4744.64392108, 37488.43146823,
        8832.32557457, 46791.81649497, 18839.48431266, 33244.72604585,
      

In [15]:
# Avaliar métricas do modelo
rmse = root_mean_squared_error(y_test,y_pred)
r2 = r2_score(y_test,y_pred)

In [16]:
# Mostrar o Erro e R2 do modelo
print(f'Root Mean Squered Error: {rmse}')
print(f'R2: {r2}')

Root Mean Squered Error: 6641.236668309879
R2: 0.7463459096735836


In [17]:
# Calcular a importância considerando os modelos do stacking regressor

importances = []

for estimator in stacking_model.estimators_:
    # Uso em modelos lineares
    if hasattr(estimator, 'coef_'):
        importances.append(np.abs(estimator.coef_))
    
    # Uso em modelos de árvore
    elif hasattr(estimator, 'feature_importances_'):
        importances.append(estimator.feature_importances_)
    
    else:
        print(f'Não foi possivel carregar a importância das variáveis do modelo {type(estimator).__name__}')



In [20]:
# Calcular a média das importâncias
importances_media = np.mean(importances, axis=0)

# Normalizar
feature_importance = importances_media/np.sum(importances_media)

In [19]:
# Obter os nomes das features
feature_names = preprocessor.get_feature_names_out()

In [21]:
df_importance = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance})

In [22]:
df_importance = df_importance.sort_values(by='Importance', ascending=True)

df_importance.head()

,Feature,Importance
5,cat__sex_male,0.006653
4,cat__sex_female,0.006654
7,cat__region_northwest,0.012387
9,cat__region_southwest,0.017066
8,cat__region_southeast,0.022745


In [23]:
# Plotar

fig = px.bar(df_importance, x='Importance', y='Feature', title='Importância das Features - Stacking regressor', orientation='h')

fig.show()

## Propriedades do modelo

In [24]:
# Mostrar evidências do modelo
# Fazer uma predição num exemplo específico

X_sample = X_test[7].reshape(1, -1)

# Predições individuais dos estimadores
lr_pred = stacking_model.named_estimators_['linear regression'].predict(X_sample)
elatic_pred = stacking_model.named_estimators_['elastic'].predict(X_sample)
tree_pred = stacking_model.named_estimators_['decision_tree'].predict(X_sample)

# Predição final com o Stacking
stacking_pred = stacking_model.predict(X_sample)

In [27]:
# Exibir os resultados
print(f'Predição do Linear Regression: {lr_pred[0]}')
print(f'Predição do ElasticNet: {elatic_pred[0]}')
print(f'Predição da Árvore de Decisão: {tree_pred[0]}')
print(f'Predição Final do Stacking (HuberRegressor): {stacking_pred[0]}')

Predição do Linear Regression: 14793.194288532943
Predição do ElasticNet: 13782.69060634142
Predição da Árvore de Decisão: 11856.4115
Predição Final do Stacking (HuberRegressor): 12191.71869976939
